# Fire Perimeter Exploration

This notebook attempts to demonstrate how large cloud datasets relevant to the disaster response community can be processed and analyzed in a Jupyter notebook.

This notebook explores near real-time fire affected areas from active wildfires from the [VIIRS Modeled Fire Perimiters dataset](https://firms.modaps.eosdis.nasa.gov/descriptions/FEDS_VIIRS_SNPP), which is updated every 12 hours and fetched from the VEDA 

This data is fetched from the [VEDA TiPG API](https://openveda.cloud/api/features/), which is the source of the [VEDA Fire Event Explorer](https://www.earthdata.nasa.gov/dashboard/tools/fire-event-explorer).

This fire extent data is paired with the [Microsoft Building Footprints](https://planetarycomputer.microsoft.com/dataset/ms-buildings#overview) dataset, a collection of almost 1 billion polygons of buildings around the world.

The notebook focuses on the US state of New Mexico, but can be adapted for other regions.

## Import Libraries

In [1]:
import requests
from owslib.ogcapi.features import Features
from pystac_client import Client

import obstore as obs
from obstore.store import from_url, AzureStore
from obstore.auth.planetary_computer import PlanetaryComputerCredentialProvider
import planetary_computer

import shapely.geometry
import mercantile

import pyarrow.parquet as pq

import deltalake
import adlfs

from lonboard import viz
import pandas as pd
import geopandas as gpd
import jupytergis

import rasterio as rs
from rasterio.plot import show

## Define our AOI

We'll be exploring data in the US state of New Mexico.

In [2]:
aoi_polygon = shapely.geometry.box("-110", "31", "-103", "38") 
aoi = ["-110", "31", "-103", "38"]

## Get Fire Perimeter data

The current vector data is hosted as a [TiPG](https://developmentseed.org/tipg/) feature (and tile) API, which is distinct from the [VEDA STAC API](https://openveda.cloud/).

This section is similar to [this notebook](https://docs.openveda.cloud/user-guide/notebooks/tutorials/mapping-fires.html).

In [3]:
VEDA_TiPG_url = "https://openveda.cloud/api/features"

Datasets distributed through OGC APIs are organized into collections. We can see the available collections with this command:

In [4]:
w = Features(VEDA_TiPG_url)
w.feature_collections()

['public.eis_fire_snapshot_perimeter_nrt',
 'public.eis_fire_lf_perimeter_nrt',
 'public.eis_fire_lf_newfirepix_nrt',
 'public.eis_fire_snapshot_newfirepix_nrt',
 'public.eis_fire_snapshot_fireline_nrt',
 'public.eis_fire_lf_fireline_nrt',
 'public.eis_fire_lf_fireline_archive',
 'public.eis_fire_lf_newfirepix_archive',
 'public.eis_fire_lf_perimeter_archive',
 'pg_temp.eis_fire_lf_perimeter_nrt_latest',
 'public.st_squaregrid',
 'public.st_hexagongrid',
 'public.st_subdivide']

The `public.eis_fire_snapshot_perimeter_nrt` collection shows the extent of active fires. We will use that as our collection.

In [5]:
perim = w.collection("public.eis_fire_snapshot_perimeter_nrt")

We can look at the variables, or queryables, that the collection has available:

In [6]:
perim_q = w.collection_queryables("public.eis_fire_snapshot_perimeter_nrt")
list(perim_q["properties"])

['geometry',
 'duration',
 'farea',
 'fireid',
 'flinelen',
 'fperim',
 'geom_counts',
 'isactive',
 'low_confidence_grouping',
 'meanfrp',
 'n_newpixels',
 'n_pixels',
 'pixden',
 'primarykey',
 'region',
 't']

We can get the items within the collection that are within our AOI, and add them to a GeoDataFrame.

In [7]:
perim_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    bbox=aoi,
    limit=1000
)

perimeters = gpd.GeoDataFrame.from_features(perim_results["features"], crs="EPSG:4326")
print("There are", len(perimeters), "active wildfire perimeters within the AOI.")

There are 78 active wildfire perimeters within the AOI.


We can quickly visualize these active wildfire perimeters with [Lonboard's](https://developmentseed.org/lonboard/latest/) `viz` function. Lonboard excels at visualizing large datasets, and the `viz` function lets us do so quickly.

In [8]:
viz(perimeters)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Buildings

Let's explore how these fires interact with our built environment. To do so, we can use [Microsoft's Building Footprints](https://planetarycomputer.microsoft.com/dataset/ms-buildings#overview) dataset, which is hosted as a STAC collection on Planetary Computer. The dataset includes almost 1 billion buildings around the world.

We can interact with the Planetary Computer STAC catalog in this notebook by loading its URL.

In [9]:
mpc_url = "https://planetarycomputer.microsoft.com/api/stac/v1/"
mpc = Client.open(mpc_url)
mpc.title

'Microsoft Planetary Computer STAC API'

To find the relevant collection, we can use the `query` parameter and search for the keyword `building`.

In [10]:
mpc_search = mpc.collection_search(
    q="building"
)

print(f"{mpc_search.matched()} collections found")

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


2 collections found


In [11]:
for collection in mpc_search.collections():
    print(collection.id)

3dep-lidar-classification
ms-buildings


`ms-buildings` is the correct collection. We can ensure that by loading the collection's title.

In [12]:
buildings_cat = mpc.get_collection("ms-buildings")
buildings_cat.title

'Microsoft Building Footprints'

The dataset is very large, and covers most of the planet. We'd like to only fetch data within our AOI.

As described in the [example notebook](https://planetarycomputer.microsoft.com/dataset/ms-buildings#Example-Notebook), the dataset is spatially partitioned using quadkeys, and is stored in [Delta format](https://docs.delta.io/latest/delta-intro.html). That means that we first should identify the quadkeys that our data exists in.

We could fetch data for all quadkeys within our AOI, but a more efficient method would be to only fetch data for quadkeys that have an active fire perimeter.

In [15]:
fire_quadkeys = set()
for geometry in perimeters.geometry:
    quadkeys = [
        int(mercantile.quadkey(tile))
        for tile in mercantile.tiles(*geometry.bounds, zooms=9)
    ]
    fire_quadkeys.update(quadkeys)

fire_quadkeys = list(fire_quadkeys)

The data can be loaded as a client with the `pystac` library.

In [ ]:
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace, # this is needed for rate limiting
)
collection = catalog.get_collection("ms-buildings") # loading the collection as before
asset = collection.assets["delta"] # loading the assets in delta format

For authentication purposes, we can store the account name and token received from fetching the collection assets...

In [19]:
storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}

...and pass them to create our delta table.

In [20]:
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

This lets us query the building dataset for only geoparquet files that cover the United States that have fire perimeters within the same quadkey.

In [22]:
file_uris = table.file_uris([("RegionName", "=", "UnitedStates"),("quadkey", "in", fire_quadkeys)])
file_uris

['az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102312/part-00005-b32140c2-71a7-4634-ad9b-e468f8e82e70.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23102330/part-00005-c55f6fcd-acbc-4139-b202-de34f44a596c.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103322/part-00033-93412344-ba19-4f70-a6bb-d89e38509ca6.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103120/part-00051-e6d7e01e-4c98-497e-aa48-314143b640ce.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23100323/part-00053-52c7e09c-d9d9-4685-b38b-c56bbc37259c.c000.snappy.parquet',
 'az://footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=UnitedStates/quadkey=23103000/part-00122-dace51aa-2205-402a-9336-3f3ee05acf35.c000.snappy.parquet'

We can then fetch the actual items (each geoparquet file) from the list of geoparquet files and store them in a geodataframe.

In [27]:
buildings = pd.concat(
    [
        gpd.read_parquet(file_uri, storage_options=storage_options)
        for file_uri in file_uris
    ]
)

We now have over 1 million buildings in a single dataframe, with their geometry, mean height, and the region and quadkey they came from.

In [33]:
buildings

,geometry,meanHeight,RegionName,quadkey
0,"POLYGON ((-108.20774 33.19588, -108.20778 33.1...",2.908320,UnitedStates,23102312
1,"POLYGON ((-107.67342 33.33829, -107.67356 33.3...",-1.000000,UnitedStates,23102312
2,"POLYGON ((-107.59869 33.27274, -107.59853 33.2...",-1.000000,UnitedStates,23102312
3,"POLYGON ((-107.64454 33.34756, -107.64446 33.3...",-1.000000,UnitedStates,23102312
4,"POLYGON ((-107.64928 33.34947, -107.6493 33.34...",-1.000000,UnitedStates,23102312
...,...,...,...,...
25120,"POLYGON ((-104.22538 32.43169, -104.22538 32.4...",2.504073,UnitedStates,23103233
25121,"POLYGON ((-104.22057 32.38902, -104.22071 32.3...",-1.000000,UnitedStates,23103233
25122,"POLYGON ((-104.2181 32.38154, -104.21826 32.38...",3.599401,UnitedStates,23103233
25123,"POLYGON ((-104.18929 32.34827, -104.18929 32.3...",-1.000000,UnitedStates,23103233


Finally, we can join the buildings and fire perimeters to see how many buildings are within an active fire perimeter.

In [34]:
buildings_in_perimiters = gpd.sjoin(buildings, perimeters, predicate='within')
buildings_in_perimiters

,geometry,meanHeight,RegionName,quadkey,index_right,duration,farea,fireid,flinelen,fperim,geom_counts,isactive,low_confidence_grouping,meanfrp,n_newpixels,n_pixels,pixden,primarykey,region,t
549,"POLYGON ((-108.09954 33.68163, -108.09958 33.6...",-1.000000,UnitedStates,23102312,25,4.0,46.295951,57474,0.000000,72.882442,None,0,0,45.843333,3,197,4.255232,CONUS|57474|2025-06-16T12:00:00,CONUS,2025-06-16T12:00:00
93,"POLYGON ((-108.16982 33.03085, -108.16967 33.0...",4.616078,UnitedStates,23102330,21,18.5,224.191592,57334,1.496575,68.072558,None,1,0,4.110000,1,1757,7.837047,CONUS|57334|2025-06-30T12:00:00,CONUS,2025-06-30T12:00:00
223,"POLYGON ((-108.14718 33.02737, -108.14718 33.0...",-1.000000,UnitedStates,23102330,21,18.5,224.191592,57334,1.496575,68.072558,None,1,0,4.110000,1,1757,7.837047,CONUS|57334|2025-06-30T12:00:00,CONUS,2025-06-30T12:00:00
381,"POLYGON ((-108.02127 32.94688, -108.02125 32.9...",3.737103,UnitedStates,23102330,21,18.5,224.191592,57334,1.496575,68.072558,None,1,0,4.110000,1,1757,7.837047,CONUS|57334|2025-06-30T12:00:00,CONUS,2025-06-30T12:00:00
473,"POLYGON ((-108.1463 33.02554, -108.14628 33.02...",4.098737,UnitedStates,23102330,21,18.5,224.191592,57334,1.496575,68.072558,None,1,0,4.110000,1,1757,7.837047,CONUS|57334|2025-06-30T12:00:00,CONUS,2025-06-30T12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10783,"POLYGON ((-104.11123 32.26805, -104.11123 32.2...",17.066706,UnitedStates,23103233,32,6.0,0.170982,57868,1.525828,1.525828,None,0,0,0.450000,1,2,11.697167,CONUS|57868|2025-06-21T00:00:00,CONUS,2025-06-21T00:00:00
19219,"POLYGON ((-104.13598 32.31467, -104.13598 32.3...",-1.000000,UnitedStates,23103233,67,0.0,0.141000,60287,1.177624,1.177624,None,1,0,0.350000,1,1,7.092199,CONUS|60287|2025-06-26T00:00:00,CONUS,2025-06-26T00:00:00
21226,"POLYGON ((-104.13596 32.31463, -104.13609 32.3...",-1.000000,UnitedStates,23103233,67,0.0,0.141000,60287,1.177624,1.177624,None,1,0,0.350000,1,1,7.092199,CONUS|60287|2025-06-26T00:00:00,CONUS,2025-06-26T00:00:00
21537,"POLYGON ((-104.13603 32.31306, -104.13589 32.3...",8.234772,UnitedStates,23103233,67,0.0,0.141000,60287,1.177624,1.177624,None,1,0,0.350000,1,1,7.092199,CONUS|60287|2025-06-26T00:00:00,CONUS,2025-06-26T00:00:00
